# **Primer Punto (Parte 1)**

 Considere los ejemplos vistos en clase, en los que se analizó la implementación de los modelos: k-nn, regresión lineal, regresión logística. Realice hiperparametrización y validación cruzada usando (i) GridSearchCV y Pipeline vs (ii) Manualmente, sin usar GridSearchCV y Pipeline (Use ciclos FOR, WHILE, etc,. . . ). Verifique que los scores obtenidos en los ítems (i)-(ii) son los mismos. Los ejercicios son los siguientes:


- Breast Cancer: (KNN, LogisticRegression) Decida cual es la métrica de mayor importancia en la aplicación de detección de cancer (métrica de negocio). Utilice esta métrica para la evaluación y selección del modelo y justifique su respuesta. Los resultados deben ser presentados usando el Cuadro 1. El estudiante que obtenga el mejor score con esta métrica, será premiado con una décima para el corte. Los datos deben ser cargados mediante el siguiente par de líneas:

In [5]:
import warnings
warnings.filterwarnings('ignore')
import mglearn
import matplotlib
import numpy as np

### Hiperparametrización y validación cruzada.
1. GridSearchCV y pipeline
2. manualmente


#### cargar los datos

In [6]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

Para los modelos que realizaremos a continuación, hemos decido que la mejor métrica de negocio en esta ocasión es *recall*, ya que, queremos limitar el npumero de falsos negativos.

#### **K-nn**


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Realizamos primeramente el modelo k-nn utilizando GridsearchCV y pipeline

In [20]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.3, random_state=10)

# Definir el pipeline
knn_pipeline = Pipeline([
    ('knn', KNeighborsClassifier())
])

# Definir los hiperparámetros a explorar
neighbors = {'knn__n_neighbors': [1, 10, 100, 1000, 10000]}

# Lista de métricas a evaluar
scoring_metrics = ['precision', 'recall', 'f1', 'roc_auc']

for metric in scoring_metrics:
    print(f"Evaluando métrica: {metric}")
    
    # Realizar la búsqueda de hiperparámetros usando GridSearchCV
    knngridsearch = GridSearchCV(knn_pipeline, neighbors, cv=5, scoring=metric)
    knngridsearch.fit(X_train, y_train)
    
    # Imprimir los mejores parámetros y el mejor score para la métrica actual
    print(f"Mejores parámetros para k-NN con {metric}: {knngridsearch.best_params_}")
    print(f"Mejor score para k-NN con {metric}: {knngridsearch.best_score_}\n")

Evaluando métrica: precision
Mejores parámetros para k-NN con precision: {'knn__n_neighbors': 10}
Mejor score para k-NN con precision: 0.9296070701784988

Evaluando métrica: recall
Mejores parámetros para k-NN con recall: {'knn__n_neighbors': 100}
Mejor score para k-NN con recall: 0.9918367346938777

Evaluando métrica: f1
Mejores parámetros para k-NN con f1: {'knn__n_neighbors': 10}
Mejor score para k-NN con f1: 0.9457994660175351

Evaluando métrica: roc_auc
Mejores parámetros para k-NN con roc_auc: {'knn__n_neighbors': 100}
Mejor score para k-NN con roc_auc: 0.9609260478384902



Luego de efectuar el modelo, obtuvimos que el mejor parámetro para k-nn era 100, con un score de 99% usando la metrica *recall*.

### **Regresión logística**

Además, tambien realizamos el modelo para la regresión logística haciendo uso de Gridsearch y pipeline.

In [21]:

regrelog_pipe = Pipeline([('lr', LogisticRegression())])

# Definir los hiperparámetros a probar
reglog = {'lr__C': [1, 10, 100, 1000, 10000]}

# Definir las métricas que se van a evaluar
scoring = {
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'AUC': 'roc_auc'
}

# Realizar GridSearchCV con múltiples métricas y refit en 'recall' (puedes cambiarlo según la métrica que prefieras)
reglog_gridsearch = GridSearchCV(regrelog_pipe, reglog, cv=5, scoring=scoring, refit='recall')

# Ajustar el modelo
reglog_gridsearch.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor score
print("Mejores parámetros para reglog:", reglog_gridsearch.best_params_)
print("Mejor score para reglog (según recall):", reglog_gridsearch.best_score_)

# Resultados de todas las métricas para los mejores parámetros
results = reglog_gridsearch.cv_results_
for metric in scoring:
    print(f"Mejor {metric}: {results[f'mean_test_{metric}'][reglog_gridsearch.best_index_]}")



Mejores parámetros para reglog: {'lr__C': 1}
Mejor score para reglog (según recall): 0.963265306122449
Mejor precision: 0.9381179796543677
Mejor recall: 0.963265306122449
Mejor f1: 0.9496007714229572
Mejor AUC: 0.9888259820057055


Obtuvimos que el mejor parámetro para la regresión logística es 1, con un score del 96% usando la métrica *recall*.

### **Modelos manuales**

##### K-nn (manual)

In [10]:
best_score = 0
for n_neighbors in [1, 10, 100, 1000, 10000]:
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)        
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring="recall") 
    score = np.mean(scores)
    if score > best_score:
        best_score = score
        best_parameters = {'n_neighbors': n_neighbors}
print("Best cross-validation accuracy: {:.8f}".format(best_score))
print("Best parameters: ", best_parameters)

Best cross-validation accuracy: 0.99183673
Best parameters:  {'n_neighbors': 100}


Luego de efectuar el modelo k-nn manual, podemos observar que obtuvimos el mismo resultado al modelo utilizando GridsearchCV.

#### reg logística (manual)

In [11]:
best_score = 0
for C in [1, 10, 100, 1000, 10000]:
    regrelogis = LogisticRegression(C = C)
    scores = cross_val_score(regrelogis, X_train, y_train, cv=5, scoring='recall')
    meanscores = np.mean(scores)
    
    if meanscores > best_score:
        best_score = meanscores
        best_parameters = {'lr__C': C }

print("Mejores parámetros para reglog:",best_parameters)
print("Mejor score para reglog:", best_score)
    

Mejores parámetros para reglog: {'lr__C': 1}
Mejor score para reglog: 0.963265306122449


Podemos observar que con el modelo de regresión logística de forma manual hemos obtenido el mismo resultado que con el modelo con el que fue utilizado el GridsearchCV. Obtuvimos que el mejor parámetro para este es 1 con un score de 96%.

## Tabla resultados

$$
\begin{array}{|l|c|c|c|c|}
\hline
\textbf{Modelo} & \textbf{Precisión} & \textbf{Recall} & \textbf{F1-Score} & \textbf{AUC} \\ \hline
\textbf{K-NN} & 0.9296070701784988 & 0.9918367346938777 & 0.9457994660175351 & 0.9609260478384902 \\ \hline
\textbf{Regresión Logística} &  0.9381179796543677 & 0.963265306122449 & 0.9496007714229572 & 0.9888259820057055 \\ \hline
\end{array}
$$

